Cryptocurrency trading bot.

In [ ]:
##%pip install python-binance==0.7.5
#%pip install --upgrade python-binance
#%pip install paramiko

## API key management.

In [ ]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
client

## Get all available pairs for information needed to trade.

In [ ]:
from cryptocurrency.exchange import Cryptocurrency_exchange
exchange = Cryptocurrency_exchange(client=client, directory='crypto_logs')
exchange_info = exchange.info
exchange_info

## Get conversion table for price conversions.

In [ ]:
from_asset = 'BDOT'
to_asset = 'BETH'

from cryptocurrency.conversion import get_shortest_pair_path_between_assets

shortest_pair_path_between_assets = \
    get_shortest_pair_path_between_assets(from_asset, to_asset, 
                                          exchange_info, 
                                          priority='fees')

shortest_pair_path_between_assets

In [ ]:
from cryptocurrency.conversion_table import get_conversion_table
conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
conversion_table

## Pre-screen buyable base assets in pairs.

In [ ]:
from cryptocurrency.conversion_table import get_new_tickers
tickers_list = get_new_tickers(conversion_table=conversion_table)
len(tickers_list)

## Locally (saving time) convert price from an asset to another using existing pairs.

In [ ]:
from cryptocurrency.trade import select_asset_with_biggest_wallet
from_asset, converted_quantity, quantity = \
    select_asset_with_biggest_wallet(client=client, 
                                     conversion_table=conversion_table, 
                                     exchange_info=exchange_info, 
                                     as_pair=True)
from_asset, converted_quantity, quantity

## Trade.

In [ ]:
import pandas as pd

def get_conversion_table_2(conversion_table_file_name='crypto_logs/conversion_table.txt'):
    return pd.read_csv(conversion_table_file_name, index_col=0)

conversion_table_file_name = 'crypto_logs/conversion_table.txt'
conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, 
                                        as_pair=True, dump_raw=True)
conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
conversion_table

In [ ]:
to_asset = 'BTC'
priority = 'fees'

from cryptocurrency.trade import trade
import pandas as pd

def get_conversion_table_2(conversion_table_file_name='crypto_logs/conversion_table.txt'):
    return pd.read_csv(conversion_table_file_name, index_col=0)

conversion_table_file_name = 'crypto_logs/conversion_table.txt'
conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                exchange_info=exchange_info, priority=priority)
from_asset = to_asset
request

## Main loop buying screened assets in pairs which are connected to the wallet holdings.

In [ ]:
sell_asset = 'BTC'
keys_file = 'contabo_keys.txt'
#crypto_output_log_screened = '~/workspace/crypto_logs/crypto_output_log_15s_screened.txt'
crypto_output_log_screened_1min = '~/workspace/crypto_logs/crypto_output_log_1min_screened.txt'
crypto_output_log_screened_30min = '~/workspace/crypto_logs/crypto_output_log_30min_screened.txt'

with open(keys_file, 'r') as f:
    password = f.readline().replace('\n', '')

import sys
import time
import pandas as pd
import paramiko

if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect('154.12.239.24', username='sam', port=22, password=password)

def get_tradable_pairs(crypto_output_log_screened='~/workspace/crypto_logs/crypto_output_log_30min_screened.txt'):
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('cat {}'.format(crypto_output_log_screened))
    tradable_pairs = pd.read_csv(StringIO(str(ssh_stdout.read().decode("utf-8"))))
    return tradable_pairs

def choose_to_asset_to_buy(from_asset, to_asset):
    tradable_pairs = get_tradable_pairs(crypto_output_log_screened=crypto_output_log_screened_30min)
    if tradable_pairs.shape[0] < 1:
        to_asset = sell_asset
    else:
        print('.', end='')
        tradable_assets_30min = set(tradable_pairs['symbol'].tolist())
        tradable_pairs = get_tradable_pairs(crypto_output_log_screened=crypto_output_log_screened_1min)
        if tradable_pairs.shape[0] < 1:
            to_asset = sell_asset
        else:
            tradable_assets_1min = set(tradable_pairs['symbol'].tolist())
            tradable_assets = list(tradable_assets_1min & tradable_assets_30min)
            if len(tradable_assets) < 1:
                to_asset = sell_asset
            else:
                latest_asset = tradable_assets[-1]
                if from_asset not in tradable_assets:
                    to_asset = latest_asset
    return to_asset

def choose_to_asset_to_sell(from_asset, to_asset):
    tradable_pairs = get_tradable_pairs(crypto_output_log_screened=crypto_output_log_screened_1min)
    if tradable_pairs.shape[0] < 1:
        to_asset = sell_asset
    else:
        print('.', end='')
        tradable_assets = list(set(tradable_pairs['symbol'].tolist()))
        latest_asset = tradable_assets[-1]
        if from_asset not in tradable_assets:
            to_asset = latest_asset
    return to_asset

def choose_to_asset(from_asset, to_asset, crypto_output_log_screened=crypto_output_log_screened_1min):
    tradable_pairs = get_tradable_pairs(crypto_output_log_screened=crypto_output_log_screened)
    if tradable_pairs.shape[0] < 1:
        to_asset = sell_asset
    else:
        print('.', end='')
        tradable_assets = list(set(tradable_pairs['symbol'].tolist()))
        latest_asset = tradable_assets[-1]
        if from_asset not in tradable_assets:
            to_asset = latest_asset
    return to_asset
    

while True:
    #if from_asset != sell_asset:
    #    to_asset = choose_to_asset_to_buy(from_asset, to_asset)
    #else:
    #    to_asset = choose_to_asset_to_sell(from_asset, to_asset)
    to_asset = choose_to_asset(from_asset, to_asset, 
                               crypto_output_log_screened=crypto_output_log_screened_30min)

    if from_asset != to_asset:
        conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
        from_asset, converted_quantity, quantity = \
            select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                             exchange_info=exchange_info)
        request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                        exchange_info=exchange_info, priority=priority)
        if request is not None:
            if to_asset != sell_asset:
                pair_bought = request['symbol']
                price_when_bought = float(request['fills'][0]['price'])
            else:
                asset_to_delete = from_asset
            from_asset = to_asset

    time.sleep(5)

ssh.close()

In [ ]:
take_profit = None
stop_loss = None
sell_asset = 'BTC'
bought_asset = None
asset_to_delete = None
keys_file = 'contabo_keys.txt'
crypto_input_log_screened = 'crypto_logs/crypto_input_log_15s_screened.txt'
crypto_output_log_screened = '~/workspace/crypto_logs/crypto_output_log_1min_screened.txt'

with open(keys_file, 'r') as f:
    password = f.readline().replace('\n', '')

from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.conversion import get_base_asset_from_pair
from cryptocurrency.exchange import Cryptocurrency_exchange

import sys
import time
import pandas as pd
import paramiko

if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect('154.12.239.24', username='sam', port=22, password=password)

client = Cryptocurrency_authenticator(use_keys=False, testnet=False).spot_client
exchange_info = Cryptocurrency_exchange(client=client, directory='crypto_logs').info

conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
#conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
from_asset, converted_quantity, quantity = \
    select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                     exchange_info=exchange_info)

while True:
    try:
        if asset_to_delete is None:
            #tradable_assets = pd.read_csv(crypto_output_log_screened, index_col=0)['base_asset']
            ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('cat {}'.format(crypto_output_log_screened))
            tradable_pairs = pd.read_csv(StringIO(str(ssh_stdout.read().decode("utf-8"))))
            tradable_assets = tradable_pairs['symbol'].apply(lambda pair: get_base_asset_from_pair(pair, 
                                                                                                   exchange_info))
            if tradable_assets.size < 1:
                to_asset = sell_asset
            else:
                tradable_assets = list(set(tradable_assets.tolist()))
                latest_asset = tradable_assets[-1]
                if from_asset not in tradable_assets:
                    to_asset = latest_asset

            if from_asset != to_asset:
                conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
                #conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
                from_asset, converted_quantity, quantity = \
                    select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                                     exchange_info=exchange_info)
                request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                                exchange_info=exchange_info, priority=priority)
                if request is not None:
                    if to_asset != sell_asset:
                        pair_bought = request['symbol']
                        price_when_bought = float(request['fills'][0]['price'])
                    else:
                        asset_to_delete = from_asset
                    from_asset = to_asset
            elif bought_asset is not None:
                conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
                #conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
                price_now = conversion_table[conversion_table['symbol'] == bought_asset]['lastPrice'].iat[0]
                percent_gain = ((price_now - price_when_bought) / price_when_bought) * 100.0
                if stop_loss is not None:
                    if percent_gain <= -stop_loss:
                        to_asset = latest_asset
                elif take_profit is not None:
                    if percent_gain >= take_profit:
                        to_asset = latest_asset
                else:
                    asset_to_delete = None
            time.sleep(2)
        else:
            input_pairs = pd.read_csv(crypto_input_log_screened, index_col=0)
            input_pairs = input_pairs[input_pairs['base_asset'] != asset_to_delete]
            input_pairs.to_csv(crypto_input_log_screened)
            asset_to_delete = None
    except (IndexError, pd.errors.EmptyDataError):
        time.sleep(0.5)
        continue

ssh.close()

In [ ]:
take_profit = None
stop_loss = None
sell_asset = 'BTC'
bought_asset = None
asset_to_delete = None
keys_file = 'contabo_keys.txt'
crypto_input_log_screened = 'crypto_logs/crypto_input_log_15s_screened.txt'
crypto_output_log_screened = '~/workspace/crypto_logs/crypto_output_log_1min_screened.txt'

with open(keys_file, 'r') as f:
    password = f.readline().replace('\n', '')

from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.conversion import get_base_asset_from_pair
from cryptocurrency.exchange import Cryptocurrency_exchange

import sys
import time
import pandas as pd
import paramiko

if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect('154.12.239.24', username='sam', port=22, password=password)

client = Cryptocurrency_authenticator(use_keys=False, testnet=False).spot_client
exchange_info = Cryptocurrency_exchange(client=client, directory='crypto_logs').info

conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
#conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
from_asset, converted_quantity, quantity = \
    select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                     exchange_info=exchange_info)

while True:
    try:
        if asset_to_delete is None:
            #tradable_assets = pd.read_csv(crypto_output_log_screened, index_col=0)['base_asset']
            ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('cat {}'.format(crypto_output_log_screened))
            tradable_pairs = pd.read_csv(StringIO(str(ssh_stdout.read().decode("utf-8"))))
            tradable_assets = tradable_pairs['symbol'].apply(lambda pair: get_base_asset_from_pair(pair, 
                                                                                                   exchange_info))
            if tradable_assets.size < 1:
                to_asset = sell_asset
            else:
                tradable_assets = list(set(tradable_assets.tolist()))
                latest_asset = tradable_assets[-1]
                if from_asset not in tradable_assets:
                    to_asset = latest_asset

            if from_asset != to_asset:
                conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
                #conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
                from_asset, converted_quantity, quantity = \
                    select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                                     exchange_info=exchange_info)
                request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                                exchange_info=exchange_info, priority=priority)
                if request is not None:
                    if to_asset != sell_asset:
                        pair_bought = request['symbol']
                        price_when_bought = float(request['fills'][0]['price'])
                    else:
                        asset_to_delete = from_asset
                    from_asset = to_asset
            elif bought_asset is not None:
                conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
                #conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
                price_now = conversion_table[conversion_table['symbol'] == bought_asset]['lastPrice'].iloc[0]
                percent_gain = ((price_now - price_when_bought) / price_when_bought) * 100.0
                if stop_loss is not None:
                    if percent_gain <= -stop_loss:
                        to_asset = latest_asset
                elif take_profit is not None:
                    if percent_gain >= take_profit:
                        to_asset = latest_asset
                else:
                    asset_to_delete = None
            time.sleep(2)
        else:
            input_pairs = pd.read_csv(crypto_input_log_screened, index_col=0)
            input_pairs = input_pairs[input_pairs['base_asset'] != asset_to_delete]
            input_pairs.to_csv(crypto_input_log_screened)
            asset_to_delete = None
    except (IndexError, pd.errors.EmptyDataError):
        time.sleep(0.5)
        continue

ssh.close()

In [ ]:
'''
take_profit = 10.0
stop_loss = None
sell_asset = 'BTC'
bought_asset = None
asset_to_delete = None
crypto_input_log_screened = 'crypto_logs/crypto_input_log_1min_screened.txt'
crypto_output_log_screened = 'crypto_logs/crypto_output_log_30min_screened.txt'

from time import sleep
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange

client = Cryptocurrency_authenticator(use_keys=False, testnet=False).spot_client
exchange_info = Cryptocurrency_exchange(client=client, directory='crypto_logs').info

#conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
from_asset, converted_quantity, quantity = \
    select_asset_with_biggest_wallet(client=client, conversion_table=conversion_table, 
                                     exchange_info=exchange_info)

while True:
    try:
        if asset_to_delete is None:
            tradable_assets = pd.read_csv(crypto_output_log_screened, index_col=0)['base_asset']
            if tradable_assets.size < 1:
                to_asset = sell_asset
            else:
                tradable_assets = list(set(tradable_assets.tolist()))
                latest_asset = tradable_assets[-1]
                if from_asset not in tradable_assets:
                    to_asset = latest_asset

            if from_asset != to_asset:
                #conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
                conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
                request = trade(client=client, to_asset=to_asset, conversion_table=conversion_table, 
                                exchange_info=exchange_info, priority=priority)
                if request is not None:
                    if to_asset != sell_asset:
                        pair_bought = request['symbol']
                        price_when_bought = float(request['fills'][0]['price'])
                    else:
                        asset_to_delete = from_asset
                    from_asset = to_asset
            elif bought_asset is not None:
                #conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, as_pair=True)
                conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
                price_now = conversion_table[conversion_table['symbol'] == bought_asset]['lastPrice'].iloc[0]
                percent_gain = ((price_now - price_when_bought) / price_when_bought) * 100.0
                if stop_loss is not None:
                    if percent_gain <= -stop_loss:
                        to_asset = latest_asset
                elif take_profit is not None:
                    if percent_gain >= take_profit:
                        to_asset = latest_asset
                else:
                    asset_to_delete = None
            sleep(5)
        else:
            input_pairs = pd.read_csv(crypto_input_log_screened, index_col=0)
            input_pairs = input_pairs[input_pairs['base_asset'] != asset_to_delete]
            input_pairs.to_csv(crypto_input_log_screened)
            asset_to_delete = None
    except (IndexError, pd.errors.EmptyDataError):
        sleep(0.5)
        continue
'''

In [ ]:
keys_file = 'contabo_keys.txt'
output_log_screened = '~/workspace/crypto_logs/crypto_output_log_1min_screened.txt'

with open(keys_file, 'r') as f:
    password = f.readline().replace('\n', '')

import sys
import time
import pandas as pd
import paramiko

if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect('154.12.239.24', username='sam', port=22, password=password)

while True:
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command('cat {}'.format(crypto_output_log_screened))

    df = pd.read_csv(StringIO(str(ssh_stdout.read().decode("utf-8"))))

    if df.shape[0] < 1:
        print('None')
    else:
        print(df)

    time.sleep(5)

ssh.close()

In [ ]:
try:
    tradable_pairs = pd.read_csv(crypto_output_log_screened, index_col=0)['symbol']
    conversion_table = get_conversion_table_2(conversion_table_file_name=conversion_table_file_name)
    tradable_assets = [get_base_asset_from_pair(tradable_pair, exchange_info=exchange_info) 
                       for tradable_pair in tradable_pairs.tolist()]
    tradable_assets = list(set(tradable_assets))
except pd.errors.EmptyDataError:
    sleep(0.5)
    tradable_assets = []
tradable_assets

In [ ]:
def get_highest_daily_volume_pair_associated_with_base_asset(base_asset, conversion_table, exchange_info):
    pairs = exchange_info[exchange_info['baseAsset'] == base_asset]['symbol'].tolist()
    filtered_pairs = conversion_table[conversion_table['symbol'].isin(pairs)]
    filtered_pairs = filtered_pairs.sort_values(by=['volume'], ascending=False)
    return filtered_pairs['symbol'].iloc[0]

pair = get_highest_daily_volume_pair_associated_with_base_asset(base_asset='LSK', 
                                                                conversion_table=conversion_table, 
                                                                exchange_info=exchange_info)

pair

In [ ]:
def get_highest_daily_volume_pair_associated_with_base_asset(base_asset, conversion_table, exchange_info):
    pairs = exchange_info[exchange_info['baseAsset'] == base_asset]['symbol'].tolist()
    filtered_pairs = conversion_table[conversion_table['symbol'].isin(pairs)]
    filtered_pairs = filtered_pairs.sort_values(by=['volume'], ascending=False)
    return filtered_pairs['symbol'].iloc[0]

crypto_input_log_screened_name = 'crypto_logs/crypto_input_log_1min_screened.txt'
dataset = pd.read_csv(crypto_input_log_screened_name, index_col=0)

dataset['base_asset'] = dataset['symbol'].apply(lambda x: get_base_asset_from_pair(x, exchange_info=exchange_info))
dataset['symbol'] = dataset['base_asset'].apply(lambda x: get_highest_daily_volume_pair_associated_with_base_asset(
                                                              base_asset=x, 
                                                              conversion_table=conversion_table, 
                                                              exchange_info=exchange_info
                                                          ))
dataset = dataset.drop_duplicates(subset=['symbol'], keep='last')
dataset